In [7]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics
import tensorflow as tf
import re

In [2]:
import numpy as np


def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

print(x_train.shape)


(3601, 500)
(1320, 500)
(3601, 500, 1)


In [8]:
from tensorflow.keras.layers import MultiHeadAttention, Embedding

In [11]:
fall_files = glob.glob('/Users/tousif/Lstm_transformer/Fall/**.xlsx')
adl_files = glob.glob('/Users/tousif/Lstm_transformer/ADL/***.xlsx')

In [12]:
all_file_path = fall_files + adl_files

In [18]:
fall_pattern = re.compile("Fall")

if fall_pattern.match(all_file_path[100]):
    print('True')
else:
    print('False')

False


In [21]:
file_len = []
trials = []
labels = []
fall_pattern = re.compile("Fall")
for file_path in all_file_path:
    
    #checking if the excel has 2 sheets or not
    label = None
    if fall_pattern.search(file_path):
        label = 1
    else:
        label = 0
                 
    if len (pd.ExcelFile(file_path).sheet_names) == 2:
        fall_df = pd.read_excel(file_path, sheet_name = -1)
        df = pd.read_excel(fall_files[1], sheet_name=-1)
        df = df.iloc[:, :-1]
        null_col = df[df.isnull().any(axis = 1)].index.to_list()
        
        #calculating how many null segments we have 
        null_col = df[df.isnull().any(axis = 1)].index.to_list()
        null_seg = len(null_col)//10
        trial_start_lst = null_col[9::10]
        trial_end_lst = null_col[10::10]
        
        for i in range(len(null_col)//10 + 1):
            trial = None
            if i == 0 :
                trial = df.iloc[0:null_col[1]-1, 3:6]
            elif i == null_seg :
                trial = df.iloc[trial_start_lst[-1]+1:, 3:6]
            else: 
                trial_end = trial_end_lst[i-1]
                trial_start = trial_start_lst[i-1] + 1
                trial = df.iloc[trial_start : trial_end-1 , 3:6].values
                trial = tf.convert_to_tensor(trial, dtype = tf.float32)
            labels.append(label)
            trials.append(trial)
trials = tf.keras.utils.pad_sequences(trials, maxlen= 512, value = float('-inf') , dtype = float, padding = 'post' )

#transposing the trials 
# trials = tf.transpose(trials, perm = [0,2,1])
# add a 

Metal device set to: Apple M1 Pro


2022-11-20 17:28:11.875107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-20 17:28:11.875737: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
trials.shape

(1020, 512, 3)

In [ ]:
#Git commit
!git add .
!git commit -m "Feat: Tranposing matrix"
!git push 



In [56]:
masking_layer = tf.keras.layers.Masking(mask_value = float('-inf'), input_shape = (512, 3))

In [53]:
masking_layer = tf.keras.layers.Embedding(input_dim = 512, output_dim = 64)

In [54]:
x = masking_layer(trials)

In [55]:
x.shape

TensorShape([460, 512, 3, 64])

In [57]:
x = masking_layer.compute_mask(trials)

In [58]:
x.shape

TensorShape([460, 512])

In [68]:
x[9]

<tf.Tensor: shape=(512,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  Tr

In [ ]:
from tensorflow.keras.layers import Add, Dense, Dropout, MultiHeadAttention, LayerNormalization, Layer, Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, Callback
from tensorflow_addons.optimizers import AdamW
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split 

import math
import wandb
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt